

```
input graph signals
1 --- 1 ---- 0 ---- 0
|     |      |      |
|     |      |      |
1 --- 1 ---- 0 ---- 0

graph labels
0 --- 1 ---- 1 ---- 0
|     |      |      |
|     |      |      |
0 --- 1 ---- 1 ---- 0

```

In [30]:
import torch
from torch import nn
import torch_geometric as pyg
import numpy as np

from torch_geometric.nn import Sequential
from torch_geometric.nn import GCNConv
from SimCKGConv import CKGConv
from tqdm import tqdm, trange
from torch_sparse import SparseTensor


## Prepare Synthetic Data

In [31]:
# Data
'''
input graph signals
1 --- 1 ---- 0 ---- 0
|     |      |      |
|     |      |      |
1 --- 1 ---- 0 ---- 0

graph labels
0 --- 1 ---- 1 ---- 0
|     |      |      |
|     |      |      |
0 --- 1 ---- 1 ---- 0
'''

adj = [
    [0, 1, 0, 0, 1, 0, 0, 0],
    [1, 0, 1, 0, 0, 1, 0, 0],
    [0, 1, 0, 1, 0, 0, 1, 0],
    [0, 0, 1, 0, 0, 0, 0, 1],
    [1, 0, 0, 0, 0, 1, 0, 0],
    [0, 1, 0, 0, 1, 0, 1, 0],
    [0, 0, 1, 0, 0, 1, 0, 1],
    [0, 0, 0, 1, 0, 0, 1, 0],
]

x = torch.Tensor([1, 1, 0, 0, 1, 1, 0, 0]).view(-1, 1)
y = torch.Tensor([0, 1, 1, 0, 0, 1, 1, 0]).view(-1, 1)
# y = torch.LongTensor([1, 0, 0, 1, 1, 0]).view(-1, 1)

adj = torch.Tensor(adj)

edge_index, edge_attr = pyg.utils.dense_to_sparse(adj)


In [32]:
deg = adj.sum(dim=1, keepdim=True)
rw = adj / deg.view(-1, 1)
out = torch.eye(adj.size(0))
rrwp = [out]
for i in range(5):
    out = out @ rw
    rrwp.append(out)
rrwp = torch.stack(rrwp, dim=-1)
rrwp_ = SparseTensor.from_dense(rrwp, has_value=True)
row, col, rrwp_val = rrwp_.coo()
rrwp_index = torch.stack([col, row], dim=0)

rrwp_val = rrwp_val * x.size(0)
pe_dim = rrwp_val.size(1)



In [33]:
rrwp_val

tensor([[8.0000, 0.0000, 3.3333, 0.0000, 2.4630, 0.0000],
        [0.0000, 4.0000, 0.0000, 3.2222, 0.0000, 2.8302],
        [0.0000, 0.0000, 1.3333, 0.0000, 1.8148, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.4444, 0.0000, 0.9753],
        [0.0000, 4.0000, 0.0000, 2.7778, 0.0000, 2.2253],
        [0.0000, 0.0000, 3.3333, 0.0000, 2.9815, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.5556, 0.0000, 1.9691],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.7407, 0.0000],
        [0.0000, 2.6667, 0.0000, 2.1481, 0.0000, 1.8868],
        [8.0000, 0.0000, 3.1111, 0.0000, 2.6790, 0.0000],
        [0.0000, 2.6667, 0.0000, 2.0741, 0.0000, 2.2058],
        [0.0000, 0.0000, 0.8889, 0.0000, 1.2099, 0.0000],
        [0.0000, 0.0000, 2.2222, 0.0000, 1.9877, 0.0000],
        [0.0000, 2.6667, 0.0000, 2.7407, 0.0000, 2.5947],
        [0.0000, 0.0000, 1.7778, 0.0000, 2.1235, 0.0000],
        [0.0000, 0.0000, 0.0000, 1.0370, 0.0000, 1.3128],
        [0.0000, 0.0000, 0.8889, 0.0000, 1.2099, 0.0000],
        [0.000

## Configuration

In [34]:
in_dim = 1
out_dim = 1
hid_dim = 1
max_ep = 200
lr = 1e-2
device = 4


## 1-layer GCN

In [35]:

loss_ls = []
acc_ls = []

for r in range(5):
    num_layers = 1
    # GCN = [(nn.Linear(in_dim, hid_dim), 'x -> x')]
    GCN = []
    for i in range(num_layers):
        GCN += [(GCNConv(hid_dim, out_dim, add_self_loops=True, normalize=True), 'x, edge_index -> x'),
                # (nn.BatchNorm1d(hid_dim), 'x -> x'),
                ]
    # GCN += [(nn.Linear(hid_dim, out_dim), 'x -> x'),
    #         (nn.Sigmoid(), 'x -> x')]
    GCN += [(nn.Sigmoid(), 'x -> x')]

    GCN = Sequential('x, edge_index', GCN)


    # ------- To Device -------
    GCN.to(device)
    opt = torch.optim.Adam(GCN.parameters(), lr=lr)
    loss_fn = nn.BCELoss()

    x, y = x.to(device), y.to(device)

    edge_index = edge_index.to(device)

    ### ----- Traing -------
    t_bar = trange(max_ep, desc='GCN', leave=True)
    for ep in t_bar:
        opt.zero_grad()
        pred = GCN.forward(x, edge_index)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        pred_ = (pred >= 0.5).type(torch.long)
        acc = torch.mean((pred_ == y).type(torch.float))
        t_bar.set_description(f'GCN: bce_loss={loss.item()}  acc={acc.item()}')


    loss_ls.append(loss.item())
    acc_ls.append(acc.item())

print(f'loss:  {np.mean(loss_ls)}$\pm$ {np.std(loss_ls)}', )
print(f'acc:  {np.mean(acc_ls)}$\pm$ {np.std(acc_ls)}', )



GCN: bce_loss=0.6917070746421814  acc=0.5: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 163.38it/s]

loss:  0.6926361203193665$\pm$ 0.0009010252144337916
acc:  0.5$\pm$ 0.0


## 1-layer CKGCN

In [36]:

loss_ls = []
acc_ls = []

for r in range(5):
    num_layers = 1
    # CKGCN = [(nn.Linear(in_dim, hid_dim), 'x -> x')]
    CKGCN = []
    for i in range(num_layers):
        CKGCN += [(CKGConv(hid_dim, out_dim, pe_dim, ffn_ratio=1., num_blocks=1, attn_dropout=0.), 'x, pe_index, pe_val -> x'),
                  # (nn.BatchNorm1d(hid_dim), 'x -> x'),
                  # (nn.GELU(), 'x -> x')
                  ]
    # CKGCN += [(nn.Linear(hid_dim, out_dim), 'x -> x'),
    #           (nn.Sigmoid(), 'x -> x')]
    CKGCN += [(nn.Sigmoid(), 'x -> x')]

    CKGCN = Sequential('x, pe_index, pe_val', CKGCN)

    # ------- To Device -------
    CKGCN.to(device)
    opt = torch.optim.Adam(CKGCN.parameters(), lr=lr)
    loss_fn = nn.BCELoss()

    x, y = x.to(device), y.to(device)

    rrwp_index, rrwp_val = rrwp_index.to(device), rrwp_val.to(device)

    ### ----- Traing -------
    t_bar = trange(max_ep, desc='CKGCN', leave=True)
    for ep in t_bar:
        opt.zero_grad()
        pred = CKGCN.forward(x, rrwp_index, rrwp_val)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        pred_ = (pred >= 0.5).type(torch.long)
        acc = torch.mean((pred_ == y).type(torch.float))
        t_bar.set_description(f'CKGCN: bce_loss={loss.item()}  acc={acc.item()}')

    loss_ls.append(loss.item())
    acc_ls.append(acc.item())

print(f'loss:  {np.mean(loss_ls)}$\pm$ {np.std(loss_ls)}', )
print(f'acc:  {np.mean(acc_ls)}$\pm$ {np.std(acc_ls)}', )




CKGCN: bce_loss=0.00017499510431662202  acc=1.0: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:00<00:00, 202.01it/s]

loss:  0.00014146362082101405$\pm$ 0.000124280314756527
acc:  1.0$\pm$ 0.0


## 1-layer CKGConv+Softmax (all positive)

In [40]:

loss_ls = []
acc_ls = []

for r in range(5):
    num_layers = 1
    # CKGCN = [(nn.Linear(in_dim, hid_dim), 'x -> x')]
    CKGCN = []
    for i in range(num_layers):
        CKGCN += [(CKGConv(hid_dim, out_dim, pe_dim, ffn_ratio=1., num_blocks=1, attn_dropout=0., softmax=True), 'x, pe_index, pe_val -> x'),
                  # (nn.BatchNorm1d(hid_dim), 'x -> x'),
                  # (nn.GELU(), 'x -> x')
                  ]
    # CKGCN += [(nn.Linear(hid_dim, out_dim), 'x -> x'),
    #           (nn.Sigmoid(), 'x -> x')]
    CKGCN += [(nn.Sigmoid(), 'x -> x')]

    CKGCN = Sequential('x, pe_index, pe_val', CKGCN)

    # ------- To Device -------
    CKGCN.to(device)
    opt = torch.optim.Adam(CKGCN.parameters(), lr=lr)
    loss_fn = nn.BCELoss()

    x, y = x.to(device), y.to(device)

    rrwp_index, rrwp_val = rrwp_index.to(device), rrwp_val.to(device)

    ### ----- Traing -------
    t_bar = trange(max_ep, desc='CKGCN', leave=True)
    for ep in t_bar:
        opt.zero_grad()
        pred = CKGCN.forward(x, rrwp_index, rrwp_val)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        pred_ = (pred >= 0.5).type(torch.long)
        acc = torch.mean((pred_ == y).type(torch.float))
        t_bar.set_description(f'CKGCN: bce_loss={loss.item()}  acc={acc.item()}')


    loss_ls.append(loss.item())
    acc_ls.append(acc.item())

print(f'loss:  {np.mean(loss_ls)}$\pm$ {np.std(loss_ls)}', )
print(f'acc:  {np.mean(acc_ls)}$\pm$ {np.std(acc_ls)}', )



CKGCN: bce_loss=0.6931471824645996  acc=0.5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 148.67it/s]


loss:  0.6931471824645996$\pm$ 0.0
acc:  0.5$\pm$ 0.0


## 1-layer CKGCN + softplus

In [41]:

loss_ls = []
acc_ls = []

for r in range(5):
    num_layers = 1
    # CKGCN = [(nn.Linear(in_dim, hid_dim), 'x -> x')]
    CKGCN = []
    for i in range(num_layers):
        CKGCN += [(CKGConv(hid_dim, out_dim, pe_dim, ffn_ratio=1., num_blocks=1, attn_dropout=0., softplus=True), 'x, pe_index, pe_val -> x'),
                  # (nn.BatchNorm1d(hid_dim), 'x -> x'),
                  # (nn.GELU(), 'x -> x')
                  ]
    # CKGCN += [(nn.Linear(hid_dim, out_dim), 'x -> x'),
    #           (nn.Sigmoid(), 'x -> x')]
    CKGCN += [(nn.Sigmoid(), 'x -> x')]

    CKGCN = Sequential('x, pe_index, pe_val', CKGCN)

    # ------- To Device -------
    CKGCN.to(device)
    opt = torch.optim.Adam(CKGCN.parameters(), lr=1e-3)
    loss_fn = nn.BCELoss()

    x, y = x.to(device), y.to(device)

    rrwp_index, rrwp_val = rrwp_index.to(device), rrwp_val.to(device)

    ### ----- Traing -------
    t_bar = trange(max_ep, desc='CKGCN', leave=True)
    for ep in t_bar:
        opt.zero_grad()
        pred = CKGCN.forward(x, rrwp_index, rrwp_val)
        loss = loss_fn(pred, y)
        loss.backward()
        opt.step()
        pred_ = (pred >= 0.5).type(torch.long)
        acc = torch.mean((pred_ == y).type(torch.float))
        t_bar.set_description(f'CKGCN: bce_loss={loss.item()}  acc={acc.item()}')


    loss_ls.append(loss.item())
    acc_ls.append(acc.item())

print(f'loss:  {np.mean(loss_ls)}$\pm$ {np.std(loss_ls)}', )
print(f'acc:  {np.mean(acc_ls)}$\pm$ {np.std(acc_ls)}', )



CKGCN: bce_loss=0.7734920978546143  acc=0.5: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:01<00:00, 172.20it/s]

loss:  0.6871560573577881$\pm$ 0.04851409215896071
acc:  0.6$\pm$ 0.1224744871391589


 ## 6-layer CKGCN